In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [1]:
orders_features = catalog.load("orders_features")
parameters = catalog.load("parameters")

2022-03-10 22:01:43,252 - kedro.io.data_catalog - INFO - Loading data from `orders_features` (PickleDataSet)...
2022-03-10 22:01:43,930 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...


In [37]:
# from hpbandster run
resultdict = catalog.load("resultdict")
resultdict.keys()
resultdict["id2config"][resultdict["incumbent"]]["config"]

{'depth': 8,
 'l2_leaf_reg': 20,
 'lr': 0.2620077982153718,
 'max_basket': 1,
 'n_min': 100,
 'sort': 'conf',
 'weight': True}

In [8]:
results_cols = catalog.load("results_cols")
enc_cols = catalog.load("enc_cols")
res_df = pd.DataFrame(results_cols)
best_cols = []
for ind in res_df.iloc[:-1].index:
    best_cols += [res_df.loc[ind][res_df.loc[ind]==max(res_df.loc[ind])].index.to_series().iloc[0]]
    # print(ind, max(res_df.loc[ind]))

In [133]:
res_df.loc[['MCC_test', 'F1_test', 'Recall_test', 'Prec_test']]

,itemID,10_m,20_m,50_m,100_m,10_m_smooth,20_m_smooth,50_m_smooth,100_m_smooth,10,...,100_3_False_lift_base,100_3_False_conf_base,100_5_True_lift_base,100_5_True_conf_base,100_5_False_lift_base,100_5_False_conf_base,100_100_True_lift_base,100_100_True_conf_base,100_100_False_lift_base,100_100_False_conf_base
MCC_test,0.306884,0.302741,0.301525,0.306654,0.304269,0.30459,0.306118,0.301672,0.302741,0.285979,...,0.301079,0.305908,0.305727,0.298762,0.301765,0.302411,0.302982,0.303248,0.302063,0.302063
F1_test,0.678212,0.682443,0.683252,0.683146,0.684735,0.685154,0.684768,0.683413,0.681844,0.674225,...,0.681473,0.682662,0.683493,0.680748,0.681286,0.679973,0.682144,0.682026,0.68102,0.68102
Recall_test,0.735387,0.757561,0.762595,0.755244,0.765152,0.766511,0.762755,0.763075,0.755084,0.746015,...,0.755763,0.754245,0.757921,0.755843,0.754085,0.747932,0.756003,0.755164,0.752607,0.752607
Prec_test,0.629286,0.620878,0.618864,0.623614,0.619613,0.61941,0.62125,0.618812,0.621555,0.61504,...,0.620482,0.623489,0.622375,0.619227,0.621305,0.623335,0.621433,0.621805,0.621868,0.621868


In [98]:
res_df[enc_cols['m_smooth']].iloc[:4]

,10_m_smooth,20_m_smooth,50_m_smooth,100_m_smooth
MCC_test,0.30459,0.306118,0.301672,0.302741
F1_test,0.685154,0.684768,0.683413,0.681844
Recall_test,0.766511,0.762755,0.763075,0.755084
Prec_test,0.61941,0.62125,0.618812,0.621555


In [95]:
res_df[enc_cols['1D']].iloc[:4]

,10,20,50,100
MCC_test,0.285979,0.287929,0.279059,0.278459
F1_test,0.674225,0.670812,0.665906,0.672716
Recall_test,0.746015,0.730553,0.722921,0.749451
Prec_test,0.61504,0.620104,0.617227,0.610235


In [96]:
res_df[enc_cols['1D_base']].iloc[:4]

,10_base,20_base,50_base,100_base
MCC_test,0.303391,0.299984,0.303987,0.302584
F1_test,0.68336,0.682853,0.685025,0.682709
Recall_test,0.760518,0.762995,0.766791,0.75888
Prec_test,0.620417,0.617946,0.619017,0.620435


In [84]:
# new_f["enc"] = "BetaLoo"
# new_f.set_index("enc", append=True, inplace = True)
# mapper = new_f.columns.to_series().str.extract('(\d+)').to_dict()[0]
# new_f.rename(mapper, inplace = True, axis=1)

In [51]:
new_f_b = res_df[enc_cols['1D_base']]
# new_f_b["enc"] = "Beta"
# new_f_b.set_index("enc", append=True, inplace = True)

In [ ]:
[int(s) for s in str.split() if s.isdigit()]

In [52]:
mapper = new_f_b.columns.to_series().str.extract('(\d+)').to_dict()[0]
new_f_b.rename(mapper, inplace = True, axis=1)

<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
/tmp/ipykernel_25083/4171924704.py:1: DeprecationWarning: invalid escape sequence \d
  mapper = new_f_b.columns.to_series().str.extract('(\d+)').to_dict()[0]
/tmp/ipykernel_25083/4171924704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_f_b.rename(mapper, inplace = True, axis=1)


In [92]:
pd.concat([, new_f_b.iloc[:4]], keys=['BetaLoo','Beta'])#.swaplevel(0,1,axis=1).sort_index(axis=1)

10        20        50       100        10  \
BetaLoo MCC_test     0.285979  0.287929  0.279059  0.278459       NaN   
        F1_test      0.674225  0.670812  0.665906  0.672716       NaN   
        Recall_test  0.746015  0.730553  0.722921  0.749451       NaN   
        Prec_test     0.61504  0.620104  0.617227  0.610235       NaN   
Beta    MCC_test          NaN       NaN       NaN       NaN  0.303391   
        F1_test           NaN       NaN       NaN       NaN   0.68336   
        Recall_test       NaN       NaN       NaN       NaN  0.760518   
        Prec_test         NaN       NaN       NaN       NaN  0.620417   

                           20        50       100  
BetaLoo MCC_test          NaN       NaN       NaN  
        F1_test           NaN       NaN       NaN  
        Recall_test       NaN       NaN       NaN  
        Prec_test         NaN       NaN       NaN  
Beta    MCC_test     0.299984  0.303987  0.302584  
        F1_test      0.682853  0.685025  0.682709  
        Recall_test  0.762995  0.766791   0.75888  
        Prec_test    0.617946  0.619017  0.620435

In [74]:
pd.concat([new_f.loc["MCC_test"], new_f_b.loc["MCC_test"]], axis=0)

10     0.285979
20     0.287929
50     0.279059
100    0.278459
10     0.303391
20     0.299984
50     0.303987
100    0.302584
Name: MCC_test, dtype: object

In [46]:
# generates double-cols
# https://stackoverflow.com/questions/45307296/join-merge-two-pandas-dataframes-and-use-columns-as-multiindex
